# 基于条件方差计算模糊度

### 计算$DOA$模糊度

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import arch
from tqdm import notebook

dataset_name = 'total'
# 创建一个迭代器，按照 chunksize 逐块读取数据
panel = pd.read_csv(f'{dataset_name}_panel.csv', parse_dates=['trade_date'], index_col=['code','trade_date'])
fama_df = pd.read_csv(f'fama_data.csv', parse_dates=['trade_date'], index_col=['MarkettypeID','trade_date'])
rf_df = pd.read_csv(f'rf_data.csv', parse_dates=['trade_date'], index_col=['Nrr1','trade_date'])

In [2]:
# 计算超额收益率

rf_df = rf_df.reset_index('Nrr1', drop=True)
rf_df = rf_df[['rf_day']]
panel = panel.join(rf_df, on='trade_date')
panel['log_ret_Premium'] = panel['log_ret'] - panel['rf_day']

In [3]:
# fama五因子模型

# P9714：沪深A股和创业板和科创板
# Portfolios1：2*3投资组合划分方法
# RiskPremium1：[市场风险溢价因子(流通市值加权)]
# SMB1 [市值因子(流通市值加权)]
# HML1 [帐面市值比因子(流通市值加权)]
# RMW1 [盈利能力因子(流通市值加权)]
# CMA1 [投资模式因子(流通市值加权)]
fama_name = ['RiskPremium1', 'SMB1', 'HML1', 'RMW1', 'CMA1']
fama_df = fama_df.loc[('P9714', slice(None)), :]
fama_df = fama_df[fama_df.Portfolios==1]
fama_df = fama_df.loc[(slice(None), slice(None)), fama_name]
fama_df = fama_df.reset_index('MarkettypeID', drop=True)
panel = panel.join(fama_df, on='trade_date')

In [4]:
# fama五因子模型的参数名
variable = ['log_ret_Premium']
variable.extend(fama_name)
panel = panel[variable]

In [5]:
code_indexs = panel.index.get_level_values(level=0).unique()[::-1]

In [ ]:
import warnings
from arch.univariate.base import DataScaleWarning
# 忽略特定的警告
warnings.filterwarnings("ignore", category=DataScaleWarning)

# 索引序列
code_indexs = panel.index.get_level_values(level=0).unique()
trade_date_num = len(panel.index.get_level_values(level=1).unique())
trade_date_series = pd.Series(panel.index.get_level_values(level=1).unique(), 
                              index = panel.index.get_level_values(level=1).unique())

IV_list = []
DOA_list = []
for code_index in notebook.tqdm(code_indexs):
    DOA_dict = {}
    IV_dict = {}
    for num in range(trade_date_num-22):
        start = trade_date_series.iloc[num]
        end = trade_date_series.iloc[num+22]
        df = panel.loc[(code_index, slice(start, end)), :]
        Y = df.iloc[:, 0]*100
        X = df.iloc[:, 1:]*100
        X = sm.add_constant(X)
        # 定义Fama-French模型
        model_ff = sm.OLS(Y, X).fit()
        ff_residuals = model_ff.resid
        # 定义GARCH模型
        model_garch = arch.arch_model(ff_residuals, vol='GARCH', p=1, q=1, dist='Normal', mean='Zero')
        results = model_garch.fit(disp=None)
        # 参数
        # print(results.params)
        omega, alpha, beta = results.params
        if 1 - alpha - beta == 0:
            IV = np.nan
        else:
            IV = omega/(1 - alpha - beta)/10000
        # 获取条件方差
        conditional_volatility = results.conditional_volatility
        # 计算模糊度DOA
        DOA = np.std(conditional_volatility)/100
        #保存
        IV_dict[(code_index, end)] = IV
        DOA_dict[(code_index, end)] = DOA

    # 保存数据
    DOA_df = pd.DataFrame(pd.Series(DOA_dict), columns=['DOA'])
    DOA_df = DOA_df.rename_axis(index=['code', 'trade_date'])
    IV_df = pd.DataFrame(pd.Series(IV_dict), columns=['IV'])
    IV_df = IV_df.rename_axis(index=['code', 'trade_date'])
    # print(DOA_df)
    DOA_df = DOA_df.join(IV_df)
    DOA_df.to_csv(f'DOA/{code_index}.csv')

In [52]:
# 将Wx整合为面板
import os
path = 'DOA/'
filesnames = os.listdir(path)
filesnames = [f for f in filesnames if f.lower().endswith(".csv")]
df_list = []
for filename in notebook.tqdm(filesnames):
    df_list.append(pd.read_csv(path + filename))

df = pd.concat(df_list, axis=0)
# 保存数据
df.set_index(['code','trade_date'], inplace=True)
df.sort_index(ascending=True, inplace=True)
df.to_csv('DOA.csv')

  0%|          | 0/2880 [00:00<?, ?it/s]

In [60]:
# 填充IV中的na

# transform用于对数据进行转换操作。主要用途之一是在分组操作中，对每个组进行独立的转换。
df.IV = df.IV.groupby('code').transform(lambda x: x.interpolate())
df.to_csv('DOA.csv')

### 计算$DOA\ W_x$

In [70]:
data = df[['DOA']].join(panel[X])

In [75]:
%%time
# 生成Wx
import numba as nb
# 定义岭回归函数
@nb.jit(nopython=True)
def Righe(df, lam=0.00000001):
    
    #函数切块
    Y = df[:, 0]
    X = df[:, 1:]
    # 标准化
    for i in range(X.shape[1]):
        X_mean = np.mean(X[:, i])
        X_var = np.var(X[:, i])
        if X_var != 0:
            X[:, i] = (X[:, i]-X_mean)/X_var
        else:
            X[:, i] = 0
            
    #增加常数列
    X = np.column_stack((X, np.ones(X.shape[0])))
    
    # 岭回归
    XTX = X.T @ X
    beta = np.linalg.inv(XTX + lam * np.identity(XTX.shape[1])) @ X.T @ Y
    uncertain = np.dot(X[-1, :], beta)
    
    return uncertain, beta[0], beta[1], beta[2], beta[3], beta[4]
    
# 自变量对DOA回归生成Wx
DOA_WxX = df[['DOA']].join(panel[X]).groupby('code').rolling(66, method='table').apply(Righe, raw=True, engine='numba')
DOA_WxX = DOA_WxX.reset_index(level=0, drop=True)
DOA_WxX.rename(columns={'DOA':'DOA_WxX'}, inplace=True)
DOA_WxX = DOA_WxX.drop(columns=X)

C:\Users\boyaw\.conda\envs\d2l2\Lib\site-packages\pandas\core\window\numba_.py:231: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'C', False, aligned=True), Array(float64, 1, 'A', False, aligned=True))
  sub_result = numba_func(window, *args)


CPU times: total: 37.3 s
Wall time: 44.7 s


In [83]:
# 自变量对IV回归生成Wx
IV_WxX = df[['IV']].join(panel[X]).groupby('code').rolling(66, method='table').apply(Righe, raw=True, engine='numba')
IV_WxX = IV_WxX.reset_index(level=0, drop=True)
IV_WxX.rename(columns={'IV':'IV_WxX'}, inplace=True)
IV_WxX = IV_WxX.drop(columns=X)

In [101]:
WxX = DOA_WxX.join(IV_WxX)
WxX.to_csv('WxX.csv')